In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy.stats.mstats import winsorize

In [ ]:
countries = {
    'Canada': {
        'property': "Real Residential Property Prices for Canada.csv",
        'unemployment': "Unemployment Rate Total in Canada.csv",
        'inflation': "Inflation, consumer prices for Canada.csv",
        'consumption': "Real Final Consumption Expenditure for Canada.csv",
        'file_type': 'csv',
        'start_date': '1970-01-01',
        'end_date': '2023-12-31'
    },
    'UK': {
        'property': "Real Residential Property Prices UK.csv",
        'unemployment': "Unemployment Rate Total in UK.csv",
        'inflation': "Inflation, consumer prices for UK.csv",
        'consumption': "Real Final Consumption Expenditure for United Kingdom.csv",
        'file_type': 'csv',
        'start_date': '1995-01-01',
        'end_date': '2023-12-31'
    },
    'USA': {
        'property': "Real Residential Property Prices for US.xlsx",
        'unemployment': "Unemployment Rate Total in US.csv",
        'inflation': "Inflation, consumer prices for United States.csv",
        'consumption': "Real Final Consumption Expenditure for US.csv",
        'file_type': 'excel',
        'start_date': '1971-01-01',
        'end_date': '2023-12-31'
    }
}

merged_datasets = {}

for country, info in countries.items():
    
    if info['file_type'] == 'csv':
        df_property = pd.read_csv(info['property'])
    else:
        df_property = pd.read_excel(info['property'], sheet_name="Quarterly")
    
    df_unemployment = pd.read_csv(info['unemployment'])
    df_inflation = pd.read_csv(info['inflation'])
    df_consumption = pd.read_csv(info['consumption'])


    datasets = [df_property, df_unemployment, df_inflation, df_consumption]
    for df in datasets:
        df.columns = df.columns.str.lower()
        if 'observation_date' in df.columns:
            df['observation_date'] = pd.to_datetime(df['observation_date'])
        elif 'date' in df.columns:
            df.rename(columns={'date': 'observation_date'}, inplace=True)
            df['observation_date'] = pd.to_datetime(df['observation_date'])


    df_property = df_property.set_index('observation_date').resample('Y').mean().reset_index()
    df_unemployment = df_unemployment.set_index('observation_date').resample('Y').mean().reset_index()
    df_inflation = df_inflation.set_index('observation_date').resample('Y').mean().reset_index()
    df_consumption = df_consumption.set_index('observation_date').resample('Y').mean().reset_index()


    start_date = info['start_date']
    end_date = info['end_date']
    
    df_property = df_property[(df_property['observation_date'] >= start_date) & (df_property['observation_date'] <= end_date)]
    df_unemployment = df_unemployment[(df_unemployment['observation_date'] >= start_date) & (df_unemployment['observation_date'] <= end_date)]
    df_inflation = df_inflation[(df_inflation['observation_date'] >= start_date) & (df_inflation['observation_date'] <= end_date)]
    df_consumption = df_consumption[(df_consumption['observation_date'] >= start_date) & (df_consumption['observation_date'] <= end_date)]

    merged = df_property.merge(df_unemployment, on='observation_date', how='inner', suffixes=('_property', '_unemployment'))
    merged = merged.merge(df_inflation, on='observation_date')
    merged = merged.merge(df_consumption, on='observation_date', suffixes=('_inflation', '_consumption'))


    merged.columns = ['Year', 'Property_Prices', 'Unemployment_Rate', 'Inflation', 'Consumption_Expenditure']
    

    merged_datasets[country] = merged


def missing_value_analysis(df, country_name):
    missing_counts = df.isnull().sum()
    missing_percent = (missing_counts / len(df)) * 100
    missing_df = pd.DataFrame({
        'Missing Count': missing_counts,
        'Missing Percentage (%)': missing_percent
    })
    print(f"\nMissing Value Analysis: {country_name}")
    print(missing_df)


for country, df in merged_datasets.items():
    missing_value_analysis(df, country)


In [ ]:

numeric_columns = ['Property_Prices', 'Unemployment_Rate', 'Inflation', 'Consumption_Expenditure']


def outlier_analysis(df, country_name):
    outlier_summary = []

    for col in numeric_columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]

        outlier_summary.append({
            'Feature': col,
            'Outlier Count': outliers.shape[0],
            'Outlier Percentage (%)': round(100 * outliers.shape[0] / df.shape[0], 2)
        })

    outlier_df = pd.DataFrame(outlier_summary)

    print(f"\nOutlier Analysis (IQR Method): {country_name}")
    print(outlier_df)


for country, df in merged_datasets.items():
    outlier_analysis(df, country)


In [ ]:
def winsorize_inflation(df, country_name):
    
    inflation_original = df['Inflation'].copy()


    df['Inflation'] = winsorize(df['Inflation'], limits=[0.05, 0.05])


    inflation_comparison = pd.DataFrame({
        'Original': inflation_original.describe(),
        'Winsorized': df['Inflation'].describe()
    })

    print(f"\nInflation Winsorization Comparison: {country_name}")
    print(inflation_comparison)

for country, df in merged_datasets.items():
    winsorize_inflation(df, country)


In [ ]:
numeric_columns = ['Inflation', 'Unemployment_Rate', 'Property_Prices', 'Consumption_Expenditure']


def correlation_analysis(df, country_name):

    correlation_matrix = df[numeric_columns].corr()


    print(f"\nCorrelation Matrix: {country_name}")
    print(correlation_matrix)

    # Plot heatmap
    plt.figure(figsize=(8, 6))
    sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", square=True)
    plt.title(f"Correlation Matrix (Heatmap) - {country_name}")
    plt.tight_layout()
    plt.show()


for country, df in merged_datasets.items():
    correlation_analysis(df, country)


In [ ]:
def scatter_plots_with_log(df, country_name):
    # Apply log transformations
    df['Log_Property_Prices'] = np.log(df['Property_Prices'])
    df['Log_Consumption_Expenditure'] = np.log(df['Consumption_Expenditure'])

 
    plot_vars = ['Log_Property_Prices', 'Unemployment_Rate', 'Log_Consumption_Expenditure']
    titles = ['Property Prices (log)', 'Unemployment Rate', 'Consumption Expenditure (log)']

    plt.figure(figsize=(15, 4))
    for i, y in enumerate(plot_vars):
        plt.subplot(1, 3, i + 1)
        sns.scatterplot(data=df, x='Inflation', y=y)
        plt.title(f'Inflation vs {titles[i]}')
        plt.xlabel('Inflation')
        plt.ylabel(titles[i])
        plt.grid(True)

    plt.suptitle(f'Inflation Relationships - {country_name}', fontsize=16, y=1.05)
    plt.tight_layout()
    plt.show()


for country, df in merged_datasets.items():
    scatter_plots_with_log(df, country)


In [ ]:
def regression_summary_analysis(df, country_name):
    X = df[['Inflation']]
    X = sm.add_constant(X)

    dependent_vars = {
        'Log_Property_Prices': df['Log_Property_Prices'],
        'Unemployment_Rate': df['Unemployment_Rate'],
        'Log_Consumption_Expenditure': df['Log_Consumption_Expenditure']
    }

    results = []

    for name, Y in dependent_vars.items():
        model = sm.OLS(Y, X, missing='drop').fit()
        results.append({
            'Dependent Variable': name,
            'Coefficient (Inflation)': model.params['Inflation'],
            'Intercept': model.params['const'],
            'R-squared': model.rsquared,
            'p-value (Inflation)': model.pvalues['Inflation']
        })


    summary_df = pd.DataFrame(results)


    print(f"\n--- Regression Summary: {country_name} ---")
    print(summary_df.round(4))

for country, df in merged_datasets.items():
    regression_summary_analysis(df, country)


In [ ]:
def regression_plots(df, country_name):
    plot_vars = [
        ('Log_Property_Prices', 'Property Prices (log)'),
        ('Unemployment_Rate', 'Unemployment Rate'),
        ('Log_Consumption_Expenditure', 'Consumption Expenditure (log)')
    ]

    plt.figure(figsize=(15, 4))

    for i, (var, label) in enumerate(plot_vars):
        plt.subplot(1, 3, i + 1)
        sns.regplot(
            data=df,
            x='Inflation',
            y=var,
            ci=None,
            line_kws={"color": "red"}
        )
        plt.title(f'Inflation vs {label}')
        plt.xlabel('Inflation')
        plt.ylabel(label)
        plt.grid(True)

    plt.suptitle(f'Relationship between Inflation and Economic Indicators - {country_name}', fontsize=16, y=1.05)
    plt.tight_layout()
    plt.show()

for country, df in merged_datasets.items():
    regression_plots(df, country)


In [ ]:
def hypothesis_testing(df, country_name):
    X = df[['Inflation']]
    X = sm.add_constant(X)

    dependent_vars = {
        'Log_Property_Prices': df['Log_Property_Prices'],
        'Unemployment_Rate': df['Unemployment_Rate'],
        'Log_Consumption_Expenditure': df['Log_Consumption_Expenditure']
    }


    hypothesis_results = []

    for name, Y in dependent_vars.items():
        model = sm.OLS(Y, X, missing='drop').fit()
        coef = model.params['Inflation']
        intercept = model.params['const']
        r_squared = model.rsquared
        p_value = model.pvalues['Inflation']

        decision = 'Reject H₀' if p_value < 0.05 else 'Fail to Reject H₀'

        hypothesis_results.append({
            'Dependent Variable': name,
            'Coefficient (Inflation)': round(coef, 4),
            'Intercept': round(intercept, 4),
            'R-squared': round(r_squared, 4),
            'p-value (Inflation)': round(p_value, 4),
            'Hypothesis Decision': decision
        })


    summary_df = pd.DataFrame(hypothesis_results)

    print(f"\nHypothesis Testing Results: {country_name}")
    print(summary_df)


for country, df in merged_datasets.items():
    hypothesis_testing(df, country)
